<a href="https://colab.research.google.com/github/relfarizi/Final_Project/blob/master/Final_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import Library

In [0]:
# Data

from google.colab import files
import pandas as pd
import numpy as np

## Data

In [0]:
#data = files.upload()

In [15]:
dataset = pd.read_csv("https://raw.githubusercontent.com/relfarizi/Final_Project/master/Data/dataset_aktif.csv?token=ALFJUNXTUYDET4JWGB6RXPK6L6XAY",sep=";")
dataset

,Molecule ChEMBL ID,Standard Type,Standard Relation,Standard Value
0,CHEMBL3781168,GI50,'>',10000
1,CHEMBL1421,GI50,'=',3
2,CHEMBL3781721,GI50,'=',61
3,CHEMBL3780586,GI50,'=',9
4,CHEMBL3780219,GI50,'=',140
...,...,...,...,...
106,CHEMBL1421,GI50,'=',14
107,CHEMBL1421,GI50,'=',3
108,CHEMBL1421,GI50,'=',1
109,CHEMBL1421,GI50,'=',8


## Probability Activity

In [31]:
mol_act = dataset.iloc[:,[0,3]]
mol_act
n = len(mol_act)
p_avg = 1/n
mol_act_dict = {}
sum_act = 0

for molekul in mol_act.values :
  a = np.exp(-molekul[1])
  sum_act  += a

for molekul in mol_act.values :
  #act = (molekul[1]*p_avg*n)/sum_act
  a = np.exp(-molekul[1])
  act = a/sum_act
  act = round(act,8)
  mol_act_dict[molekul[0]] = [molekul[1],act]

print(mol_act_dict)

{'CHEMBL3781168': [10000, 0.0], 'CHEMBL1421': [8, 0.00063516], 'CHEMBL3781721': [66, 0.0], 'CHEMBL3780586': [15, 5.8e-07], 'CHEMBL3780219': [7, 0.00172655], 'CHEMBL3781013': [570, 0.0], 'CHEMBL3781859': [1200, 0.0], 'CHEMBL3781577': [1849, 0.0], 'CHEMBL255863': [25, 0.0], 'CHEMBL3780622': [6, 0.00469325], 'CHEMBL3781512': [15, 5.8e-07], 'CHEMBL3780441': [42, 0.0], 'CHEMBL3780983': [5, 0.01275759], 'CHEMBL3780835': [1290, 0.0], 'CHEMBL3781199': [5200, 0.0], 'CHEMBL1214141': [14, 1.57e-06], 'CHEMBL3780326': [100, 0.0], 'CHEMBL2403816': [4300, 0.0], 'CHEMBL3780177': [1380, 0.0], 'CHEMBL3781685': [2043, 0.0], 'CHEMBL3780940': [590, 0.0], 'CHEMBL3780377': [983, 0.0], 'CHEMBL2403814': [4100, 0.0], 'CHEMBL3781253': [50, 0.0], 'CHEMBL3780021': [20, 0.0], 'CHEMBL3780040': [31, 0.0], 'CHEMBL3781590': [1300, 0.0], 'CHEMBL3780544': [55, 0.0], 'CHEMBL3780370': [1100, 0.0], 'CHEMBL3780190': [2900, 0.0], 'CHEMBL3781353': [628, 0.0], 'CHEMBL3780628': [124, 0.0], 'CHEMBL3781900': [36, 0.0], 'CHEMBL3780

## For git

In [0]:
#!git init

#!git config --global user.name “relfarizi”

#!git remote add origin https://github.com/relfarizi/Final_Project.git

#!git add .

#!git commit -m "Probability Activity"

#!git push

#!git status